# Graph matrix parametrization 

##### Dimensionality

Применяем лемму 4 и получаем различные выражения для углов, выражения с тремя углами раскрываются неоднозначно, поэтому там нужно будет рассмотреть все варианты. Таким образом получаем уравнение связи $A \varphi = c$ и уравнения $B \varphi < b$, задающий политоп. Необходимо установить, при каких вариантах раскрытия неоднозначных элементов матрицы внутренность политопа непуста. Делаем это решая задачу оптимизации.

### Trial 3 (the dumbest version)

In [6]:
import cvxpy as cp
import multiprocessing as mp
from tqdm import tqdm 
from tqdm.contrib.itertools import product
import os
import datetime

from graph_utils import *

In [11]:
ALL_GRAPHS = get_graphs()

In [12]:
ALL_GRAPHS = {'dupl':ALL_GRAPHS['dupl']}

In [14]:
def process(arg):

  # явно можно переделать на спарс матрицы и таким образом увеличить возможный размер загружаемого чанка

  totalA = np.array(A + arg[0])
  totalB = np.array([1 for _ in range(len(A))] + arg[1])

  prob = cp.Problem(cp.Maximize(func), [totalA @ x == totalB, B @ x <= c])

  prob.solve(solver="ECOS", max_iters=500, verbose=False, 
             abstol=5e-1, 
             reltol=5e-1, 
             abstol_inacc=5e-1, 
             feastol=1, 
             feastol_inacc=5e-1, 
             reltol_inacc=1)
             
  #prob.solve(solver="SCS", max_iters=500, verbose=True, eps=1e-8)

  if prob.value != -np.inf:
    return [np.linalg.matrix_rank(totalA), arg[2]]

  return []

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [15]:
if not os.path.isdir("graphData"):
    os.mkdir("graphData")

for name, g in ALL_GRAPHS.items():

  I = find_stable_sets(g)
  A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

  B, c = build_inequalities(num_params)

  x = cp.Variable(num_params)
  func = cp.sum(cp.log( c - B@x ))
  
  if not os.path.isdir("graphData/{}".format(name)):
    os.mkdir("graphData/{}".format(name))
    
  # если взять SCS solver, то всё feasable!
  # chunksize кажется особо не влияет на скорость, но влияет на занимаемую память
  # 3 граф обрабатывается быстрее всех
  # !!!! нужно оценить, какой прирост может дать переход на улучшенный алгоритм
  # !!!! запустить у себя на компе не оптимизированную версию - пускай считается, пока оптимизируем

  # два графа опять пустые, зато big_triag пофиксился - оказалось, что он был задан неверно. но эти вроде норм заданы...
  
  # я не задаю нулевую точку - это тоже может влиять на сходимость: в этих солверах нельзя их задать
  
  # SCS всегда выдаёт feasible. Если поставить константную функцию, то ECOS тоже всё считает feasible - 
  # это странно, т.к. по идеи от неё эта штука не должа зависеть. 

  with mp.Pool(processes=os.cpu_count()) as pool:

    for res in tqdm(pool.imap(process, build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True
                                                                         ), chunksize=1000)):
      if len(res) != 0:
        res[0] = num_params - res[0]
        print("hello there")
        print(res[0])
        #with open("graphData/{}/{}".format(name, str(datetime.datetime.now())), "w+") as outfile:
        #  outfile.write("\n".join(map(str, res)))

16it [00:00, 145.53it/s]

hello there
6
hello there
5
hello there
7
hello there
6
hello there
7
hello there
6
hello there
8
hello there
7
hello there
6
hello there
5
hello there
7
hello there
6
hello there
6
hello there
5
hello there
7
hello there
6
